In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import numpy.random as rnd
from sklearn import decomposition
import pickle
from tensorflow.keras.datasets import mnist

In [ ]:
# load data và reshape nó về dạng đơn
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape)
x_train = x_train.reshape(-1,784).astype('float32')/255.0
x_test = x_test.reshape(-1,784).astype('float32')/255.0
print(x_test.shape)
print(x_train.shape)

11493376/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)
(10000, 784)
(60000, 784)


In [ ]:
# tạo 1 biến e nhỏ để tránh trường hợp nhân với 0
e = 1e-9

imag_mean = np.mean(x_train,axis = 0, keepdims=True)
imag_std = np.std(x_train,axis = 0, keepdims=True)
print(imag_mean.shape)
print(imag_std.shape)
x_train -= imag_mean;    x_train /= (imag_std + e)   
x_test -= imag_mean;    x_test /= (imag_std + e)
print(x_train.shape)

(1, 784)
(1, 784)
(60000, 784)


In [ ]:
m, n = x_train.shape
sicma = 1/m*np.dot(x_train.T,x_train)
U, S, VH = np.linalg.svd(sicma)
ei = np.diag(S)
k = 100
print('Retained information: ', np.sum(ei[1:k])/np.sum(ei))
Uk = U[:,1:k]
Z = np.dot(x_train,Uk)

Retained information:  0.6494713


In [ ]:
n_components = 128
pca = decomposition.PCA(n_components=n_components)
pca.fit(x_train)
Z = pca.transform(x_train)
print('Retained ratio: ', np.sum(pca.explained_variance_ratio_))

Retained ratio:  0.76303077


In [ ]:
import tensorflow as tf
from tensorflow.python.eager.context import graph_mode
# xây dụng model 
model = tf.keras.models.Sequential()
#tạo ra 2 lớp của model với 128 điểm  
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
#tạo lớp cuối vói 10 điểm 
model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))

In [ ]:
# compile model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x_train, y_train, epochs=3)

Epoch 1/3
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2236 - accuracy: 0.9355
Epoch 2/3
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0943 - accuracy: 0.9715
Epoch 3/3
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0722 - accuracy: 0.9779


In [ ]:
#loss function and accuracy function
val_loss, val_acc = model.evaluate(x_test, y_test)
print(val_acc)

313/313 [==============================] - 0s 1ms/step - loss: 440.2401 - accuracy: 0.9734
0.9733999967575073
